In [46]:
from utils.bigdata_a3_utils import *
import pandas as pd
import os
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
# from utils.preprocessing import *
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\saeed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saeed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saeed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
os.makedirs("custom_dataset", exist_ok=True)

In [48]:
MAX_ROWS = 10000

In [49]:
reviews_csv_path = "custom_dataset/reviews.csv"
metadata_csv_path = "custom_dataset/metadata.csv"

In [50]:
# # for i, category in VALID_CATEGORIES:
# for i, category in enumerate(tqdm(VALID_CATEGORIES, desc="Processing categories")):
#     print(f"Processing {category}...")

#     try:
#         review_dataset = load_dataset(
#             "McAuley-Lab/Amazon-Reviews-2023", 
#             f"raw_review_{category}", 
#             trust_remote_code=True
#         )

#         review_df = pd.DataFrame(review_dataset['full'][:MAX_ROWS])

#         review_df["category"] = category 
        
#         mode = 'w' if i == 0 else 'a'
#         header = i == 0

#         review_df.to_csv(reviews_csv_path, mode=mode, header=header, index=False)

#         print(f"Appended {len(review_df)} rows of {category} to {reviews_csv_path}.")
#     except Exception as e:
#         print(f"Error loading dataset for {category}: {e}")
    
#     try:
#         meta_dataset = load_dataset(
#             "McAuley-Lab/Amazon-Reviews-2023", 
#             f"raw_meta_{category}", 
#             trust_remote_code=True
#         )
        
#         # Convert to pandas DataFrame and limit to MAX_ROWS
#         meta_df = pd.DataFrame(meta_dataset['full'][:MAX_ROWS])

#         meta_df['category'] = category

#         mode = 'w' if i == 0 else 'a'
#         header = i == 0

#         meta_df.to_csv(metadata_csv_path, mode=mode, header=header, index=False)
#         print(f"Loaded {len(meta_df)} rows of {category} metadata.")
#     except Exception as e:
#         print(f"Error loading dataset for {category}: {e}")

In [51]:
first_review_df = None
first_meta_df = None

# Process each category
for category in tqdm(VALID_CATEGORIES, desc="Processing categories"):
    print(f"\nProcessing category: {category}")
    
    # Process review data
    try:
        # Use streaming mode with iterable dataset
        review_dataset = load_dataset(
            "McAuley-Lab/Amazon-Reviews-2023", 
            f"raw_review_{category}", 
            trust_remote_code=True,
            split="full",
            streaming=True
        )
        
        # Create an iterator for the dataset
        review_iter = iter(review_dataset)
        
        # Collect reviews into a list (up to MAX_ROWS)
        reviews = []
        count = 0
        
        for item in review_iter:
            if count >= MAX_ROWS:
                break
                
            # Create a new dictionary instead of modifying the item in-place
            review_item = dict(item)
            review_item['category'] = category
            reviews.append(review_item)
            count += 1
            
            # Process in batches to conserve memory
            if len(reviews) >= 1000 or count >= MAX_ROWS:
                review_df = pd.DataFrame(reviews)
                
                # Check if this is the first batch we're writing
                if first_review_df is None:
                    review_df.to_csv(reviews_csv_path, mode='w', header=True, index=False)
                    first_review_df = True
                else:
                    review_df.to_csv(reviews_csv_path, mode='a', header=False, index=False)
                
                # Clear the batch
                reviews = []
        
        print(f"  ✓ Appended {count} reviews from {category} to {reviews_csv_path}")
    except Exception as e:
        print(f"  ✗ Error processing reviews for {category}: {str(e)}")
    
    # Process metadata
    try:
        # Use streaming mode with iterable dataset
        meta_dataset = load_dataset(
            "McAuley-Lab/Amazon-Reviews-2023", 
            f"raw_meta_{category}", 
            trust_remote_code=True,
            split="full",
            streaming=True
        )
        
        # Create an iterator for the dataset
        meta_iter = iter(meta_dataset)
        
        # Collect metadata into a list (up to MAX_ROWS)
        metadata = []
        count = 0
        
        for item in meta_iter:
            if count >= MAX_ROWS:
                break
                
            # Create a new dictionary instead of modifying the item in-place
            meta_item = dict(item)
            meta_item['category'] = category
            metadata.append(meta_item)
            count += 1
            
            # Process in batches to conserve memory
            if len(metadata) >= 1000 or count >= MAX_ROWS:
                meta_df = pd.DataFrame(metadata)
                
                # Check if this is the first batch we're writing
                if first_meta_df is None:
                    meta_df.to_csv(metadata_csv_path, mode='w', header=True, index=False)
                    first_meta_df = True
                else:
                    meta_df.to_csv(metadata_csv_path, mode='a', header=False, index=False)
                
                # Clear the batch
                metadata = []
        
        print(f"  ✓ Appended {count} metadata records from {category} to {metadata_csv_path}")
    except Exception as e:
        print(f"  ✗ Error processing metadata for {category}: {str(e)}")

print(f"\nAll categories processed!")
print(f"Review data saved to: {reviews_csv_path}")
print(f"Metadata saved to: {metadata_csv_path}")

Processing categories:   0%|          | 0/34 [00:00<?, ?it/s]


Processing category: All_Beauty
  ✓ Appended 10000 reviews from All_Beauty to custom_dataset/reviews.csv


Processing categories:   3%|▎         | 1/34 [00:10<06:02, 10.98s/it]

  ✓ Appended 10000 metadata records from All_Beauty to custom_dataset/metadata.csv

Processing category: Amazon_Fashion
  ✓ Appended 10000 reviews from Amazon_Fashion to custom_dataset/reviews.csv


Processing categories:   6%|▌         | 2/34 [00:20<05:14,  9.84s/it]

...

  ✓ Appended 10000 metadata records from Toys_and_Games to custom_dataset/metadata.csv

Processing category: Video_Games
  ✓ Appended 10000 reviews from Video_Games to custom_dataset/reviews.csv


Processing categories:  97%|█████████▋| 33/34 [04:59<00:08,  8.80s/it]

  ✓ Appended 10000 metadata records from Video_Games to custom_dataset/metadata.csv

Processing category: Unknown
  ✓ Appended 10000 reviews from Unknown to custom_dataset/reviews.csv


Processing categories: 100%|██████████| 34/34 [05:06<00:00,  9.00s/it]

  ✓ Appended 10000 metadata records from Unknown to custom_dataset/metadata.csv

All categories processed!
Review data saved to: custom_dataset/reviews.csv
Metadata saved to: custom_dataset/metadata.csv


In [52]:
delete_cache_directory()

[INFO] Deleting Hugging Face cache at: C:\Users\saeed\.cache\huggingface\datasets
[SUCCESS] Cache directory deleted.


In [53]:
def mergedatasets():
    reviews = pd.read_csv("custom_dataset/reviews.csv")
    metadata = pd.read_csv("custom_dataset/metadata.csv")

    merged = pd.merge(reviews, metadata, on= 'parent_asin', how='inner', suffixes=('_review', '_metadata'))
    return merged

In [54]:
merged = pd.DataFrame()
merged = mergedatasets()

C:\Users\saeed\AppData\Local\Temp\ipykernel_25088\3673331556.py:3: DtypeWarning: Columns (6,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("custom_dataset/metadata.csv")


In [55]:
merged.shape

(40278, 27)

In [56]:
merged.columns

Index(['rating', 'title_review', 'text', 'images_review', 'asin',
       'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'category_review', 'main_category',
       'title_metadata', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images_metadata', 'videos', 'store',
       'categories', 'details', 'bought_together', 'subtitle', 'author',
       'category_metadata'],
      dtype='object')

In [57]:
merged = merged[merged['text'].str.strip().astype(bool)]

In [58]:
merged.shape

(40272, 27)

In [59]:
merged = merged.drop_duplicates(subset=['user_id', 'asin', 'text'], keep='first')

In [60]:
merged.shape

(40187, 27)

In [61]:
def extract_brand(row):
    # Check if store exists and is not empty after stripping
    store_value = row.get('store')
    if isinstance(store_value, str) and store_value.strip():
        return store_value.strip()  # Return store if it has a non-empty value
    
    # Try to extract brand from details
    elif row.get('details'):
        # Use a regex pattern similar to your original function
        match = re.search(r"Brand[:\s\-]*([A-Za-z0-9&\s]+)", row['details'], re.IGNORECASE)
        if match:
            return match.group(1).strip()  # Return the brand found in details
    
    # Default fallback
    return 'Unknown'

In [62]:
merged.loc[:, 'brand'] = merged.apply(extract_brand, axis=1)

In [64]:
merged.loc[:, 'review_length'] = merged['text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

In [68]:
merged.head(1)

,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,videos,store,categories,details,bought_together,subtitle,author,category_metadata,brand,review_length
0,3.0,Meh,These were lightweight and soft but much too s...,[{'small_image_url': 'https://m.media-amazon.c...,B088SZDGXG,B08BBQ29N5,AEYORY2AVPMCPDV57CE337YU5LXA,1634275259292,0,True,...,"{'title': ['She loved them !', 'NEW FAUX 24 in...",Niseyo,[],"{""Brand"": ""Niseyo"", ""Extension Length"": ""24 In...",NaN,NaN,NaN,All_Beauty,Niseyo,32


In [66]:
analysis_path = "custom_dataset/analysis.csv"

In [69]:
needed_columns = {
                'main_category': 'category',
                'brand': 'brand',
                'title_metadata': 'item',
                'rating': 'rating',
                'text': 'reviewText',
                'review_length': 'text_length',
                'timestamp': 'timestamp',
                'rating_number': 'numRating',
                'average_rating': 'average_rating',
                'helpful_vote':'total_votes'
                }

merged = merged[list(needed_columns.keys())]
merged.rename(columns=needed_columns, inplace=True)
merged.shape

C:\Users\saeed\AppData\Local\Temp\ipykernel_25088\585969604.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns=needed_columns, inplace=True)


(40187, 10)

In [70]:
merged.to_csv(analysis_path, index=False)

In [71]:
df = pd.read_csv(analysis_path)

In [72]:
df.head(1)

,category,brand,item,rating,reviewText,text_length,timestamp,numRating,average_rating,total_votes
0,All Beauty,Niseyo,Niseyo new Faux Locs 24 Inch Crochet Hair 6 Pa...,3.0,These were lightweight and soft but much too s...,32,1634275259292,62.0,4.3,0


In [73]:
df.head(1)

,category,brand,item,rating,reviewText,text_length,timestamp,numRating,average_rating,total_votes
0,All Beauty,Niseyo,Niseyo new Faux Locs 24 Inch Crochet Hair 6 Pa...,3.0,These were lightweight and soft but much too s...,32,1634275259292,62.0,4.3,0


In [74]:
print(df.shape)
print(df.isnull().sum()) 

(40187, 10)
category          2699
brand                0
item                 4
rating               0
reviewText          12
text_length          0
timestamp            0
numRating          286
average_rating       0
total_votes          0
dtype: int64


## Custom popularity metric
Popularity Score (PS)
PS= 𝛼 × Number of Reviews + 𝛽 × Average Rating+ 𝛾 × Total Votes

Where:

Number of Reviews = total reviews per item per month (or week).

Average Rating = mean of ratings per item per month.

Total Votes = sum of votes (helpfulness) on the reviews per item per month.

α, β, γ = tunable weights based on what you want to emphasize more.

In [75]:
alpha = 0.6
beta = 0.3
gamma = 0.1

In [76]:
df["popularity_score"] = ((alpha * df["numRating"]) + (beta * df["average_rating"]) + (gamma * df["total_votes"]))

In [77]:
df.head(1)

,category,brand,item,rating,reviewText,text_length,timestamp,numRating,average_rating,total_votes,popularity_score
0,All Beauty,Niseyo,Niseyo new Faux Locs 24 Inch Crochet Hair 6 Pa...,3.0,These were lightweight and soft but much too s...,32,1634275259292,62.0,4.3,0,38.49


In [78]:
reviews_to_clean = df["reviewText"]

In [79]:
reviews_to_clean.head(1)

0    These were lightweight and soft but much too s...
Name: reviewText, dtype: object

In [80]:
def clean_html(text):
    # Remove URLs
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub('<.*?>+', '', text)
    return text

In [81]:
def tokenize_df(text):
    if isinstance(text, str):  # If the input is a string
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token.isalpha()]  # Keep only alphabetic tokens
        return tokens
    elif isinstance(text, (int, float)):  # If the input is an integer or float
        return text  # Return the number as is
    else:
        return []

In [82]:
def remove_stopwords(tokens):
    stop_words = (stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

In [83]:
def lemmatize_text(tokens):
    if tokens is None:
        return []  # Return an empty list if tokens is None
    lemma = WordNetLemmatizer()
    lemma_tokens = [lemma.lemmatize(word) for word in tokens if word is not None]  # Check for None
    return lemma_tokens

In [84]:
def clean_text(text):
    if not isinstance(text, str):  # Check if the input is not a string
        return ''  # Return an empty string or handle it as needed
    text = clean_html(text)
    tokens = tokenize_df(text)
    tokens = [token.lower() for token in tokens]
    tokens = remove_stopwords(tokens)
    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]
    tokens = lemmatize_text(tokens)
    return ' '.join(tokens)

In [85]:
cleaned_reviews = reviews_to_clean.apply(clean_text)

In [86]:
cleaned_reviews = cleaned_reviews.apply(lambda x: re.sub(r"\s+", " ", str(x)).strip() if isinstance(x, (str, float, int)) else x)

In [87]:
df.rename(columns={'reviewText': 'cleanedText'}, inplace=True)

In [88]:
df.columns

Index(['category', 'brand', 'item', 'rating', 'cleanedText', 'text_length',
       'timestamp', 'numRating', 'average_rating', 'total_votes',
       'popularity_score'],
      dtype='object')

In [89]:
df["cleanedText"] = cleaned_reviews

In [90]:
df["brand"] = df["brand"].apply(clean_text)

In [91]:
df["brand"]

0                 niseyo
1               bali boo
2                 iryasa
3             fine touch
4                  annie
              ...       
40182             gagowa
40183              pepsi
40184    annie homegrown
40185                kar
40186          funkyfish
Name: brand, Length: 40187, dtype: object

In [92]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [95]:
df.head(1)

,category,brand,item,rating,cleanedText,text_length,timestamp,numRating,average_rating,total_votes,popularity_score
0,All Beauty,niseyo,Niseyo new Faux Locs 24 Inch Crochet Hair 6 Pa...,3.0,lightweight soft much small liking would prefe...,32,2021-10-15 05:20:59.292,62.0,4.3,0,38.49


In [94]:
df.to_csv("csvs/AnalysisDataset.csv")